# Project 2

In [1]:
import torch
torch.set_grad_enabled(False)

In [2]:
class Module(object):
    def __init__(self):
        self.parameters = []
        
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return self.parameters